In [13]:
import torch
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [14]:
data_dir = '../data/'
val_dir = data_dir + 'images/valid/'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_set = ImageFolder(val_dir, transform=transform, target_transform=None)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

In [17]:
model = torch.load('../models/ResNet50_pretrained-epoch4-batch_size32-lr0.001.pt')
model.eval();

In [ ]:
pred = []
true = []
with torch.no_grad():
    for images, labels in val_loader:
        output = model(images)
        _, predicted = torch.max(output, 1)
        pred.extend(predicted.tolist())
        true.extend(labels.tolist())

accuracy = (true == pred).sum() / len(true)
print(f'Test Accuracy: {accuracy:.2%}')